In [24]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import functools
import os
import time

texts:
* The Trial: http://www.gutenberg.org/cache/epub/7849/pg7849.txt
* Metamorphosis: http://www.gutenberg.org/files/5200/5200-h/5200-h.htm
* A Hunger Artist: https://archive.org/details/AHungerArtistByFranzKafka
* The Castle: https://archive.org/details/in.ernet.dli.2015.149543/page/n11

In [87]:
filenames = ['5200-h.txt', '7849-8.txt', 'the castle.txt', 'the hunger artist.txt']
with open('texts/combined.txt', 'w') as outfile:
    for fname in filenames:
        with open('texts/'+fname) as infile:
            for line in infile:
                outfile.write(line)

In [88]:
text = open('texts/combined.txt', 'rb').read().decode(encoding='utf-8')

Preparing dataset

In [89]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

In [90]:
vocab = sorted(set(text))
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])
seq_length = 100
examples_per_epoch = len(text)//seq_length
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
dataset = sequences.map(split_input_target)

In [91]:
BATCH_SIZE = 64
steps_per_epoch = examples_per_epoch//BATCH_SIZE

BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [92]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

In [93]:
rnn = functools.partial(tf.keras.layers.GRU, recurrent_activation='sigmoid')

In [94]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, 
                              batch_input_shape=[batch_size, None]),
    rnn(rnn_units,
        return_sequences=True, 
        recurrent_initializer='glorot_uniform',
        stateful=True),
    tf.keras.layers.Dense(vocab_size)
  ])
    return model

In [95]:
model = build_model(
  vocab_size = len(vocab), 
  embedding_dim=embedding_dim, 
  rnn_units=rnn_units, 
  batch_size=BATCH_SIZE)

In [96]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (64, None, 256)           28160     
_________________________________________________________________
gru_6 (GRU)                  (64, None, 1024)          3935232   
_________________________________________________________________
dense_6 (Dense)              (64, None, 110)           112750    
Total params: 4,076,142
Trainable params: 4,076,142
Non-trainable params: 0
_________________________________________________________________


In [97]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [41]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)") 
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.173542


In [43]:
model.compile(optimizer = tf.train.AdamOptimizer(), loss = loss)

In [44]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints_shakespeare'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [45]:
EPOCHS=3

In [46]:
history = model.fit(dataset.repeat(), epochs=EPOCHS, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback])


Epoch 1/3
173/174 [============================>.] - ETA: 42s - loss: 2.6915 WARNING:tensorflow:From /Applications/anaconda/lib/python3.6/site-packages/tensorflow/python/keras/engine/network.py:1436: update_checkpoint_state (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.train.CheckpointManager to manage checkpoints rather than manually editing the Checkpoint proto.
174/174 [==============================] - 7375s 42s/step - loss: 2.6882
Epoch 2/3
 64/174 [==========>...................] - ETA: 2:08:42 - loss: 2.0484

KeyboardInterrupt: 

In [47]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints_shakespeare/ckpt_1'

In [48]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [49]:
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
gru_2 (GRU)                  (1, None, 1024)           3935232   
_________________________________________________________________
dense_2 (Dense)              (1, None, 65)             66625     
Total params: 4,018,497
Trainable params: 4,018,497
Non-trainable params: 0
_________________________________________________________________


In [50]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 1000

    # Converting our start string to numbers (vectorizing) 
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperatures results in more predictable text.
    # Higher temperatures results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1.0

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a multinomial distribution to predict the word returned by the model
        predictions = predictions / temperature
        predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()

        # We pass the predicted word as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [51]:
print(generate_text(model, start_string=u"ROMEO: "))

Instructions for updating:
Use tf.random.categorical instead.
ROMEO: Hyach, and wombne
To gone, fo, my hosh,
Ant Jor hill hald emare hy thew tied net ke pond thawe now a bayt,
Hos, Thak hourisspord-rer: an owhel of uit Torky an the prace to wich huns, Rordeg:
Low him enom hom her sour thee soomase wiof net: trather here thay my it he cimpent that inenthe to surnal steine perse,s an tove harquen zeca mos:
And ay you hat ins fit, a lly hereneert:
sie a soolo.

Pat whave oo soulle mimy mod.

CIRUSTASN:
I llath ald hiw, ul with the t ush and me mom, pofnes thot thind thoul's; nots.
Whimky youg in.
What my ppives, con-the cheas
tham is gor.

TLORINAANTKS:
I sere wimt
The ut houwing crowe, ald breide beretar firt os me the wize soll'nd forat the':
Worly, has ir eard' thens ard anchibes beeres'd
To be'vestin walleang tay soop of ardeglreds?

KINCEO:
Yome what whe hecr dith thou
Precase?
Paw not that pugnebyour, to the sed wires:
Me me,
Loreat.

JUMENET:
Go, Ill hiags by bor to maa- thas his a